In [82]:
import datetime
import ee
import geemap
import colorcet as cc
from data_utils.pygeoboundaries import get_adm_ee
from google.cloud import storage
import json
from data_utils.monitor_tasks import monitor_tasks

In [2]:
ee.Authenticate()

True

In [3]:
cloud_project = "hotspotstoplight"

In [4]:
ee.Initialize(project=cloud_project)

In [76]:
place_name = "Costa Rica"

scale = 90

snake_case_place_name = place_name.replace(" ", "_").lower()

aoi = get_adm_ee(territories=place_name, adm="ADM0")
bbox = aoi.geometry().bounds()

In [77]:
# Applies scaling factors.
def apply_scale_factors(image):
    # Scale and offset values for optical bands
    optical_bands = image.select("SR_B.").multiply(0.0000275).add(-0.2)

    # Scale and offset values for thermal bands
    thermal_bands = image.select("ST_B.*").multiply(0.00341802).add(149.0)

    # Add scaled bands to the original image
    return image.addBands(optical_bands, None, True).addBands(thermal_bands, None, True)


# Function to Mask Clouds and Cloud Shadows in Landsat 8 Imagery
def cloud_mask(image):
    # Define cloud shadow and cloud bitmasks (Bits 3 and 5)
    cloud_shadow_bitmask = 1 << 3
    cloud_bitmask = 1 << 5

    # Select the Quality Assessment (QA) band for pixel quality information
    qa = image.select("QA_PIXEL")

    # Create a binary mask to identify clear conditions (both cloud and cloud shadow bits set to 0)
    mask = (
        qa.bitwiseAnd(cloud_shadow_bitmask)
        .eq(0)
        .And(qa.bitwiseAnd(cloud_bitmask).eq(0))
    )

    # Update the original image, masking out cloud and cloud shadow-affected pixels
    return image.updateMask(mask)

In [78]:
from google.cloud import storage
import json

bucket_name = f"hotspotstoplight_heatmapping"
directory_name = f"data/{snake_case_place_name}/inputs/"

storage_client = storage.Client(project=cloud_project)
bucket = storage_client.bucket(bucket_name)
blob = bucket.blob(directory_name)
blob.upload_from_string(
    "", content_type="application/x-www-form-urlencoded;charset=UTF-8"
)

In [79]:
def export_ndvi_min_max(
    year, bbox, scale, gcs_bucket, snake_case_place_name, file_prefix="ndvi_min_max"
):
    try:
        startDate = ee.Date.fromYMD(year, 1, 1)
        endDate = ee.Date.fromYMD(year, 12, 31)

        # Filter the collection for the given year and bounds
        imageCollection = (
            ee.ImageCollection("LANDSAT/LC08/C02/T1_L2")
            .filterBounds(bbox)
            .filterDate(startDate, endDate)
            .map(apply_scale_factors)
            .map(cloud_mask)
        )

        # Calculate NDVI for the entire collection
        ndviCollection = imageCollection.map(
            lambda image: image.normalizedDifference(["SR_B5", "SR_B4"]).rename("NDVI")
        )

        # Reduce the collection to get min and max NDVI values
        ndvi_min = ndviCollection.reduce(ee.Reducer.min()).reduceRegion(
            reducer=ee.Reducer.min(), geometry=bbox, scale=scale, maxPixels=1e9
        )
        ndvi_max = ndviCollection.reduce(ee.Reducer.max()).reduceRegion(
            reducer=ee.Reducer.max(), geometry=bbox, scale=scale, maxPixels=1e9
        )

        # Create a feature to export
        feature = ee.Feature(
            None,
            {
                "ndvi_min": ndvi_min.get("NDVI_min"),
                "ndvi_max": ndvi_max.get("NDVI_max"),
            },
        )

        # Create and start the export task with the specified fileNamePrefix
        task = ee.batch.Export.table.toCloudStorage(
            collection=ee.FeatureCollection([feature]),
            description=f"{file_prefix}_{year}",
            bucket=gcs_bucket,
            fileNamePrefix=f"data/{snake_case_place_name}/inputs/{file_prefix}_{year}",
            fileFormat="CSV",
        )
        task.start()

        # Print statements confirming the task has started
        print(f"Starting export task for NDVI min/max values of year {year}.")

        # Return the task object
        return task

    except Exception as e:
        print(f"An error occurred while starting the export task for year {year}: {e}")
        return None

In [80]:
from google.cloud import storage
import csv
from io import StringIO


def download_ndvi_values_from_gcs(
    cloud_project, bucket_name, snake_case_place_name, year
):
    storage_client = storage.Client(project=cloud_project)
    bucket = storage_client.bucket(bucket_name)

    # Correctly define the blob's name to include the full path
    blob_name = f"data/{snake_case_place_name}/inputs/ndvi_min_max_{year}.csv"
    blob = bucket.blob(blob_name)

    # Download the data as a string
    ndvi_data_csv = blob.download_as_string()

    # Parse the CSV data
    ndvi_data = csv.reader(StringIO(ndvi_data_csv.decode("utf-8")))
    rows = list(ndvi_data)

    # Adjust based on the actual structure of your CSV
    # Assuming the first row after the header contains NDVI min and the second row contains NDVI max
    # Note: Ensure the row indexing matches your CSV's structure; this example assumes row 1 is headers
    ndvi_min = rows[1][1]  # Access the first column in the second row (actual data)
    ndvi_max = rows[1][2]  # Access the first column in the third row (actual data)

    return float(ndvi_min), float(ndvi_max)

In [109]:
def process_year(year, ndvi_min, ndvi_max):
    # Define the start and end dates for the year
    startDate = ee.Date.fromYMD(year, 1, 1)
    endDate = ee.Date.fromYMD(year, 12, 31)

    # Import and preprocess Landsat 8 imagery for the year
    imageCollection = (
        ee.ImageCollection("LANDSAT/LC08/C02/T1_L2")
        .filterBounds(bbox)
        .filterDate(startDate, endDate)
        .map(apply_scale_factors)
        .map(cloud_mask)
    )

    # Function to calculate LST for each image in the collection
    def calculate_lst(image):
        # Calculate Normalized Difference Vegetation Index (NDVI)
        ndvi = image.normalizedDifference(["SR_B5", "SR_B4"]).rename("NDVI")

        # Use the passed ndvi_min and ndvi_max directly instead of calculating them
        # Convert them to ee.Number since they are likely passed as Python primitives
        ndvi_min_ee = ee.Number(ndvi_min)
        ndvi_max_ee = ee.Number(ndvi_max)

        # Fraction of Vegetation (FV) Calculation
        fv = (
            ee.Image()
            .expression(
                "(ndvi - ndvi_min) / (ndvi_max - ndvi_min)",
                {"ndvi": ndvi, "ndvi_max": ndvi_max_ee, "ndvi_min": ndvi_min_ee},
            )
            .pow(2)
            .rename("FV")
        )

        # Emissivity Calculation
        em = fv.multiply(ee.Number(0.004)).add(ee.Number(0.986)).rename("EM")

        # Select Thermal Band (Band 10) and Rename It
        thermal = image.select("ST_B10").rename("thermal")

        # Land Surface Temperature (LST) Calculation
        lst = thermal.expression(
            "(TB / (1 + (0.00115 * (TB / 1.438)) * log(em))) - 273.15",
            {
                "TB": thermal.select("thermal"),  # Select the thermal band
                "em": em,  # Assign emissivity
            },
        ).rename("LST")

        return lst

    # Apply the calculate_lst function to each image in the collection
    lstCollection = imageCollection.map(calculate_lst)

    # Create a binary image for each image in the collection where 1 indicates LST >= 33 and 0 otherwise
    hotDaysCollection = lstCollection.map(lambda image: image.gte(33))

    # Sum all the binary images in the collection to get the total number of hot days in the year
    hotDaysYear = hotDaysCollection.sum()

    landcover = ee.Image("ESA/WorldCover/v100/2020").select("Map").clip(bbox)

    dem = ee.ImageCollection("projects/sat-io/open-datasets/FABDEM").mosaic().clip(bbox)

    image_for_sampling = (
        landcover.rename("landcover")
        .addBands(dem.rename("elevation"))
        .addBands(ee.Image.pixelLonLat())
        .addBands(hotDaysYear.rename("hot_days"))
    )

    print("Sampling image band names", image_for_sampling.bandNames().getInfo())

    return image_for_sampling

In [83]:
def file_exists_in_gcs(bucket_name, blob_name, cloud_project):
    """Check if a file exists in Google Cloud Storage."""
    storage_client = storage.Client(project=cloud_project)
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(blob_name)
    return blob.exists()

In [84]:
export_tasks = []

print("Starting NDVI min/max export tasks...")

for year in range(2018, 2024):
    # Define the blob's name for the CSV file
    blob_name = f"hotspotstoplight_heatmapping/data/{snake_case_place_name}/inputs/ndvi_min_max_{year}.csv"

    # Check if the file already exists
    if not file_exists_in_gcs(bucket_name, blob_name, cloud_project):

        task = export_ndvi_min_max(
            year,
            bbox,
            scale,
            bucket_name,
            snake_case_place_name,
            file_prefix="ndvi_min_max",
        )
        if task is not None:
            export_tasks.append(task)
        else:
            print(f"Failed to initiate export task for year: {year}")
    else:
        print(f"CSV file for year {year} already exists. Skipping export.")

if export_tasks:
    print(f"{len(export_tasks)} export tasks have been initiated. Monitoring tasks...")
    monitor_tasks(export_tasks)
    print("All export tasks have completed.")
else:
    print("No new export tasks were initiated.")

Starting NDVI min/max export tasks...
Starting export task for NDVI min/max values of year 2018.
Starting export task for NDVI min/max values of year 2019.
Starting export task for NDVI min/max values of year 2020.
Starting export task for NDVI min/max values of year 2021.
Starting export task for NDVI min/max values of year 2022.
Starting export task for NDVI min/max values of year 2023.
6 export tasks have been initiated. Monitoring tasks...
Monitoring tasks...
Task LUIF5XQWX2PB7L2GZ2PB5PGV completed successfully.
Task VSQVTKLHNF6UPX425QKI76DU completed successfully.
Task CDOIMRDGEY2GQ2LSJ3F36AVJ completed successfully.
Task 2MQUNLT3D6ARKRLU2AHOGELB completed successfully.
Task QQGSAQBHHZSKMTCLILO5KSEK is READY.
Task WLYNHU5NTLT6IKSZ7KJ2DJSD is READY.
Task QQGSAQBHHZSKMTCLILO5KSEK completed successfully.
Task WLYNHU5NTLT6IKSZ7KJ2DJSD completed successfully.
All tasks have been processed.
All export tasks have completed.


In [110]:
# Initialize an empty Python list to hold the processed ee.Image objects
processed_images = []

print("Starting to process and download NDVI values...")

# After ensuring all export tasks are completed, proceed to download and process
for year in range(2018, 2024):
    print(f"Downloading NDVI values for year: {year}")
    ndvi_min, ndvi_max = download_ndvi_values_from_gcs(
        cloud_project, bucket_name, snake_case_place_name, year
    )

    # Print the downloaded NDVI values
    # print(f"NDVI values for {year} - Min: {ndvi_min}, Max: {ndvi_max}")

    print(f"Processing data for year: {year}")
    image_for_sampling = process_year(year, ndvi_min, ndvi_max)

    processed_images.append(image_for_sampling)
    print(f"Completed processing for year: {year}")

print("Converting processed images into an ee.ImageCollection...")
# Convert the list of ee.Image objects into an ee.ImageCollection
final_image_collection = ee.ImageCollection.fromImages(processed_images)

print("Processing complete.")

Starting to process and download NDVI values...
Processing data for year: 2018
Sampling image band names ['landcover', 'elevation', 'longitude', 'latitude', 'hot_days']
Completed processing for year: 2018
Processing data for year: 2019
Sampling image band names ['landcover', 'elevation', 'longitude', 'latitude', 'hot_days']
Completed processing for year: 2019
Processing data for year: 2020
Sampling image band names ['landcover', 'elevation', 'longitude', 'latitude', 'hot_days']
Completed processing for year: 2020
Processing data for year: 2021
Sampling image band names ['landcover', 'elevation', 'longitude', 'latitude', 'hot_days']
Completed processing for year: 2021
Processing data for year: 2022
Sampling image band names ['landcover', 'elevation', 'longitude', 'latitude', 'hot_days']
Completed processing for year: 2022
Processing data for year: 2023
Sampling image band names ['landcover', 'elevation', 'longitude', 'latitude', 'hot_days']
Completed processing for year: 2023
Converting

In [98]:
print(final_image_collection.size().getInfo())

6


In [104]:
image = final_image_collection.filter(ee.Filter.calendarRange(2023, 2023, "year"))

vizParams = {"min": 0, "max": 35, "palette": cc.fire}
Map1 = (
    geemap.Map()
)  # Assuming you're using geemap or similar in Jupyter, or use Earth Engine code editor's Map.addLayer

for year in range(2018, 2024):
    image = final_image_collection.filter(
        ee.Filter.calendarRange(year, year, "year")
    ).median()
    if image:
        vizParams = {
            "min": 0,
            "max": 35,
            "bands": "hot_days",
        }  # Adjust visualization parameters as needed
        Map1.addLayer(image, vizParams, f"Image {year}")
    else:
        print(f"No image for year {year}")

Map1.centerObject(image, zoom=10)  # Adjust zoom as needed
Map1

TypeError: 'NoneType' object is not subscriptable

In [56]:
# Define the number of samples per year
num_samples_per_year = 25000 // (2024 - 2018)

# Initialize an empty FeatureCollection to aggregate the samples
samples_feature_collection = ee.FeatureCollection([])

# Iterate over each image in the final_image_collection
for year in range(2018, 2024):
    try:
        # Get the processed image for the given year from the ImageCollection
        processed_image = final_image_collection.filter(
            ee.Filter.calendarRange(year, year, "year")
        ).first()

        print("Attempting to sample from year:", year)
        print("Band names (uncomputed):", processed_image.bandNames().getInfo())

        # Check if the image exists to avoid sampling from a non-existent image
        if processed_image:
            # Directly sample the processed image
            sample = processed_image.sample(
                region=bbox,
                scale=scale,
                numPixels=num_samples_per_year,
                seed=0,
                geometries=True,  # Set to True if you need geometries for visualization
            )

            # Aggregate the samples into a FeatureCollection
            samples_feature_collection = samples_feature_collection.merge(sample)
        else:
            print(f"No processed image available for year {year}.")

    except Exception as e:
        print(f"Skipping year {year} due to an error: {e}")

# The samples_feature_collection now contains the aggregated samples from each year
training_sample = samples_feature_collection

Attempting to sample from year: 2018
Skipping year 2018 due to an error: Image.bandNames: Parameter 'image' is required.
Attempting to sample from year: 2019
Skipping year 2019 due to an error: Image.bandNames: Parameter 'image' is required.
Attempting to sample from year: 2020
Band names (uncomputed): ['landcover', 'elevation', 'longitude', 'latitude', 'hot_days']
Attempting to sample from year: 2021
Skipping year 2021 due to an error: Image.bandNames: Parameter 'image' is required.
Attempting to sample from year: 2022
Skipping year 2022 due to an error: Image.bandNames: Parameter 'image' is required.
Attempting to sample from year: 2023
Skipping year 2023 due to an error: Image.bandNames: Parameter 'image' is required.


In [50]:
# Split the data into training and testing
training_sample = training_sample.randomColumn()
training = training_sample.filter(ee.Filter.lt("random", 0.7))
testing = training_sample.filter(ee.Filter.gte("random", 0.7))

# Train the Random Forest regression model
inputProperties = ["longitude", "latitude", "landcover", "elevation"]
numTrees = 10  # Number of trees in the Random Forest
regressor = (
    ee.Classifier.smileRandomForest(numTrees)
    .setOutputMode("REGRESSION")
    .train(training, classProperty="hot_days", inputProperties=inputProperties)
)

In [57]:
# Filter the collection for images from 2020
image_2020_collection = final_image_collection.filter(
    ee.Filter.calendarRange(2020, 2020, "year")
)

# Take the first image from the filtered collection as the recent_image
recent_image = image_2020_collection.first()

# Proceed with the classification
predicted_image = recent_image.select(inputProperties).classify(regressor)

# Calculate the difference
difference = (
    recent_image.select("hot_days").subtract(predicted_image).rename("difference")
)

In [ ]:
# Assuming 'max_lst' is your actual maximum LST image and 'predicted_image' contains the predictions
# Calculate the squared difference between actual and predicted LST
squared_difference = (
    recent_image.select("hot_days")
    .subtract(predicted_image)
    .pow(2)
    .rename("difference")
)

# Reduce the squared differences to get the mean squared difference over your area of interest (aoi)
mean_squared_error = squared_difference.reduceRegion(
    reducer=ee.Reducer.mean(),
    geometry=bbox,
    scale=scale,  # Adjust scale to match your dataset's resolution
    maxPixels=1e14,
)

# Calculate the square root of the mean squared error to get the RMSE
rmse = mean_squared_error.getInfo()["difference"] ** 0.5

print("RMSE:", rmse)

In [58]:
vizParams = {"min": 0, "max": 35, "palette": cc.fire}

m = geemap.Map()
m.centerObject(aoi, 8)
m.add("basemap_selector")
m.add("layer_manager")
m.addLayer(recent_image.select("hot_days").clip(aoi), vizParams, "Actual Max LST")
m.addLayer(predicted_image.clip(aoi), vizParams, "Predicted LST")
m.addLayer(difference, {"min": -10, "max": 10, "palette": cc.cwr}, "Difference")
m

EEException: Image.sample: Parameter 'image' is required.